In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)
# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# this should import glob, os, and many other standard libs
from src.utils.Notebook_imports import *
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
# local imports
from src.utils.Utils_io import Console_and_file_logger, init_config
from src.visualization.Visualize import show_2D_or_3D

# import external libs
import cv2
import pandas as pd

EXPERIMENT = 'temp/PhaseRegression-tests'

# ------------------------------------------generator hyperparameters
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 2 # 32, 64, 24, 16, 1 for 3D use: 4
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
DIM = [10, 64, 64] # network input params for spacing of 3, (z,y,x)
T_SHAPE = 35
SPACING = [10, 4, 4] # if resample, resample to this spacing, (z,y,x)
# maybe we can drop this channel?
#IMG_CHANNELS = 1 # Currently our model needs that image channel
MASK_VALUES = [1, 2, 3]  #channel order: Background, RV, MYO, LV
MASK_CLASSES = len(MASK_VALUES) # no of labels
M_POOL = [2, 2, 2]# size of max-pooling used for downsampling and upsampling
F_SIZE = [3, 3, 3] # conv filter size
#MASKING_IMAGE = False # make curently no sense for regression
#MASKING_VALUES = [2] 
DF_META = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase'
# We define 5 target phases and a background phase for the pad/empty volumes 
PHASES = len(['ED#', 'MS#', 'ES#', 'PF#', 'MD#'])+1
TARGET_SMOOTHING = True
SMOOTHING_KERNEL_SIZE = 10
SMOOTHING_LOWER_BORDER = 0.2
SMOOTHING_UPPER_BORDER = 0.4
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = True # a compose of 2D augmentation (grid distortion, 90degree rotation, brightness and shift)
AUGMENT_PROB = 1
SHUFFLE = True
RESAMPLE = True
SCALER = 'MinMax' # MinMax Standard or Robust
DEPTH = 2
Console_and_file_logger(EXPERIMENT, logging.INFO)
config = init_config(config=locals(), save=True)
print(config)
logging.info('Is built with tensorflow: {}'.format(tf.test.is_built_with_cuda()))
logging.info('Visible devices:\n{}'.format(tf.config.list_physical_devices()))
logging.info('Local devices: \n {}'.format(device_lib.list_local_devices()))

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models
['/gpu:0', '/gpu:1']


2021-02-12 10:23:50,855 INFO -------------------- Start --------------------
2021-02-12 10:23:50,855 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-02-12 10:23:50,856 INFO Log file: ./logs/temp/PhaseRegression-tests.log
2021-02-12 10:23:50,856 INFO Log level for console: INFO
2021-02-12 10:23:50,857 INFO Is built with tensorflow: True
2021-02-12 10:23:50,920 INFO Visible devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


{'GPU_IDS': '0,1', 'GPUS': ['/gpu:0', '/gpu:1'], 'SEED': 42, 'EXPERIMENT': 'temp/PhaseRegression-tests', 'BATCHSIZE': 2, 'GENERATOR_WORKER': 2, 'DIM': [10, 64, 64], 'T_SHAPE': 35, 'SPACING': [10, 4, 4], 'MASK_VALUES': [1, 2, 3], 'MASK_CLASSES': 3, 'M_POOL': [2, 2, 2], 'F_SIZE': [3, 3, 3], 'DF_META': '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase', 'PHASES': 6, 'TARGET_SMOOTHING': True, 'SMOOTHING_KERNEL_SIZE': 10, 'SMOOTHING_LOWER_BORDER': 0.2, 'SMOOTHING_UPPER_BORDER': 0.4, 'BORDER_MODE': 4, 'IMG_INTERPOLATION': 1, 'MSK_INTERPOLATION': 0, 'AUGMENT': True, 'AUGMENT_PROB': 1, 'SHUFFLE': True, 'RESAMPLE': True, 'SCALER': 'MinMax', 'DEPTH': 2, 'CONFIG_PATH': 'reports/config/temp/PhaseRegression-tests', 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/temp/PhaseRegression-tests', 'MODEL_PATH': 'models/temp/PhaseRegression-tests', 'HISTORY_PATH': 'reports/history/temp/PhaseRegression-tests'}


2021-02-12 10:23:51,594 INFO Local devices: 
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 408720628556924694
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9705235050141190702
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 382194999192547380
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8732741489714593185
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22791942400
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12242450687857974990
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23561743872
locality {
  bus_id: 

In [2]:
# zip the phase files in the following structure:
#[[patient1_phase1, patient1_phase2, patient1_phase3, patient1_phase4, patient1_phase5],[patient2_phase1...] ...]

images = sorted(glob.glob('/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/*clean.nrrd'))
print(len(images))

278


In [3]:
# heck if we find each patient in the corresponding dataframe
import re
METADATA_FILE = config.get('DF_META', '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase')
df = pd.read_csv(METADATA_FILE)
DF_METADATA = df[['patient', 'ED#', 'MS#', 'ES#', 'PF#', 'MD#']]


for x in images:
    try:
        patient_str = re.search('-(.{8})_', x).group(1).upper()
        
        assert(len(patient_str) == 8), 'matched patient ID from the phase sheet has a length of: {}'.format(len(patient_str))
        # returns the indices in the following order: 'ED#', 'MS#', 'ES#', 'PF#', 'MD#'
        # reduce by one, as the indexes start at 0, the excel-sheet at 1
        ind = DF_METADATA[DF_METADATA.patient.str.contains(patient_str)][['ED#', 'MS#', 'ES#', 'PF#', 'MD#']]
        indices = ind.values[0].astype(int) -1
        
    except Exception as e:
        logging.info(patient_str)
        logging.info(ind)
        logging.info('indices: \n{}'.format(indices))

In [4]:
@interact
def search_p_in_df(search_str = ''):
    return DF_METADATA[DF_METADATA['patient'].str.contains(search_str)]

interactive(children=(Text(value='', description='search_str'), Output()), _dom_classes=('widget-interact',))

In [5]:
# initialise a generator with these files
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import PhaseRegressionGenerator
#config['SHUFFLE'] = False
#config['AUGMENT'] = False
config['RESAMPLE'] = True
batch_generator = PhaseRegressionGenerator(images, images, config=config)
val_config = config.copy()
val_config['AUGMENT'] = False
validation_generator = PhaseRegressionGenerator(images, images , config=val_config)

2021-02-12 10:23:52,104 INFO Create DataGenerator
2021-02-12 10:23:52,106 INFO Datagenerator created with: 
 shape: [10, 64, 64]
 spacing: [10, 4, 4]
 batchsize: 2
 Scaler: MinMax
 Images: 278 
 Augment: True 
 Thread workers: 2
2021-02-12 10:23:52,106 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-02-12 10:23:52,120 INFO [0.2  0.25 0.3  0.35 0.4  1.   0.4  0.35 0.3  0.25 0.2 ]
2021-02-12 10:23:52,121 INFO Create DataGenerator
2021-02-12 10:23:52,122 INFO Datagenerator created with: 
 shape: [10, 64, 64]
 spacing: [10, 4, 4]
 batchsize: 2
 Scaler: MinMax
 Images: 278 
 Augment: False 
 Thread workers: 2
2021-02-12 10:23:52,122 INFO No augmentation
2021-02-12 10:23:52,136 INFO [0.2  0.25 0.3  0.35 0.4  1.   0.4  0.35 0.3  0.25 0.2 ]


In [6]:
# visualise one batch
@interact_manual
def select_image_in_batch(batch = (0,len(batch_generator), 1), 
                          im = (0,BATCHSIZE- 1, 1), 
                          slice_n=(1,11), 
                          save=False, 
                          filepath='data/temp/', 
                          filename='temp_x.npy',
                         show_=True):
    
    import time
    global inputs, outputs
    t1 = time.time()
    inputs, outputs = batch_generator.__getitem__(batch)
    logging.info('Preprocessing took: {:0.3f}'.format(time.time() - t1))
    print(inputs.shape, outputs.shape)
    print('selected batch : ' + str(batch))
    if show_:
    
        selected_input = inputs[im]
        selected_output = outputs[im]

        logging.debug('pad: \n{}'.format(selected_output))

        list(map(lambda x: show_2D_or_3D(img=x, interpol='bilinear',dpi=100,f_size=(5,5)), selected_input[::slice_n]))
        plt.show()
        info('-'*100)
        #list(map(lambda x: show_2D_or_3D(img=x[::slice_n], interpol='bilinear',dpi=100,f_size=(5,5)), selected_output))

        """
            show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
        plt.show()
        """
        plt.hist(selected_input.flatten(), bins=np.linspace(0.1,1,20))
        plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))
    



interactive(children=(IntSlider(value=69, description='batch', max=139), IntSlider(value=0, description='im', …

In [ ]:
for b in range(len(batch_generator)):
    print(b)
    select_image_in_batch(batch=b, slice_n=5, show_=False) 

0


2021-02-12 10:23:52,538 INFO Preprocessing took: 0.330


(2, 35, 10, 64, 64, 1) (2, 35, 6)
selected batch : 0
1


2021-02-12 10:23:53,017 INFO Preprocessing took: 0.479


(2, 35, 10, 64, 64, 1) (2, 35, 6)
selected batch : 1
2


...

2021-02-12 10:23:57,680 INFO Preprocessing took: 0.433


(2, 35, 10, 64, 64, 1) (2, 35, 6)
selected batch : 13
14


2021-02-12 10:23:58,086 INFO Preprocessing took: 0.406


(2, 35, 10, 64, 64, 1) (2, 35, 6)
selected batch : 14
15


2021-02-12 10:23:58,877 INFO Preprocessing took: 0.790


In [ ]:
from src.models.Models import create_PhaseRegressionModel
model = create_PhaseRegressionModel(config)

In [ ]:
model.summary(line_length=150)

In [ ]:
initial_epoch = 0

results = model.fit(
    x=batch_generator,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=200,
    callbacks = get_callbacks(config, valid_generator),
    steps_per_epoch = len(batch_generator),
    initial_epoch=initial_epoch,
    max_queue_size=20,
    workers=2,
    use_multiprocessing=False,
    verbose=1)

# Generator tests

In [ ]:
temp = outputs[0][:]
plt.imshow(temp)

In [ ]:
t = np.concatenate([temp, temp])
print(t.shape)
first, second = np.split(t, indices_or_sections=2)
print(first.shape)
temp = np.maximum(first,second)
print(temp.shape)

In [ ]:
nonempty_timesteps = (np.sum(temp,axis=1)==0).astype(int)
nonempty_timesteps.shape
nonempty_timesteps

In [ ]:
nonempty_timesteps[:,None].shape


In [ ]:
np.concatenate([temp, nonempty_timesteps[:,None]], axis=1).shape

In [ ]:
temp = outputs[0][:]
print(temp.shape)
print(temp)
print(temp.sum(axis=0))
print(temp.T)

In [ ]:
onehot = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])

In [ ]:
onehot.shape

In [ ]:
onehot_t = onehot.T
onehot_t.shape
onehot_t

In [ ]:
print(np.argmax(onehot, axis=1))
print(np.argmax(onehot_t, axis=0))

In [ ]:
# iterate over the phases j
# linspace from i to i+1 from indices
temp = onehot_t[:,0].astype(float)
i = 0
j = 3 + 1
temp[i:j] = np.linspace(i,j,j-i)
temp

In [ ]:
onehot_t = onehot_t.astype(float)
indices = np.argmax(onehot_t, axis=0)
for i in range(onehot_t.shape[1]) :
    start = indices[i]
    # make it a ring
    second = (i+1)%len(indices)
    end = indices[second]
    onehot_t[start:end,i] = np.linspace(1,0,int(end-start))
onehot_t
             
    

In [ ]:
# smooth the one-hot vectors along the indexes
# each phase has a small range of valid indexes with a convolved smoothing
import sys
temp = np.copy(outputs[0])
kernelsize = 5
lower = 0.1
upper = 0.5
# create a kernel with linearly increasing, than one, than decreasing smoothing
kernel = np.concatenate([np.linspace(lower,upper,kernelsize//2), [1], np.linspace(upper,lower,kernelsize//2)])
print('Kernel: {}'.format(kernel))
for idx in range(temp.shape[0]):
    print(temp[idx])
    smoothed = np.convolve(temp[idx],kernel, mode='same')
    print('convolved: {}'.format(smoothed))
    smoothed = smoothed/(sum(smoothed) + sys.float_info.epsilon)
    print('normalised: {}'.format(smoothed))
    temp[idx] = smoothed


In [ ]:
plt.imshow(temp)

In [ ]:
plt.imshow(temp.T)

In [ ]:
# load phase indicies from the df
print(len(images))
patient_temp = images[0]
# search for the 8 digits of the patient ID which should have a underscore in before and than a minus
patient_str = re.search('-(.{8})_', patient_temp).group(1)
df = pd.read_csv('/mnt/ssd/data/gcn/02_imported_4D_unfiltered_old/SAx_3D_dicomTags_phase')
df = df[['patient', 'ED#', 'MS#', 'ES#', 'PF#', 'MD#']]
df[df.patient.str.contains(patient_str)][['ED#', 'MS#', 'ES#', 'PF#', 'MD#']].values[0].astype(int) -1

In [ ]:
import re
a = get_patient(patient_temp)

re.search('-(.{8})_', patient_temp).group(1)

In [ ]:
%%timeit
temp.mean()

In [ ]:
%%timeit
np.mean(temp)

In [ ]:
np.linspace(-1,1,10)